### Naive Semantic Chunker

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import re

from rich.console import Console
from rich.table import Table
from rich.panel import Panel

console = Console()

In [2]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [3]:
def naive_semantic_chunking(
    text: str,
    threshold: float = 0.75,
    model_name: str = "all-MiniLM-L6-v2"
):
    """
    Level 0 — Naive baseline semantic chunking.
    DO NOT use in production. This is for understanding only.
    """

    console.rule("[bold red]Level 0 — Naive Semantic Chunking (What NOT to do)")

    # ------------------------------------------------------------------
    # Step 1: Sentence splitting (very naive)
    # ------------------------------------------------------------------
    sentences = [
        s.strip()
        for s in re.split(r"(?<=[.!?])\s+", text)
        if s.strip()
    ]

    # ------------------------------------------------------------------
    # Step 2: Embeddings (single sentence, no windowing)
    # ------------------------------------------------------------------
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences, normalize_embeddings=False)

    # ------------------------------------------------------------------
    # Step 3: Show sentences + embeddings
    # ------------------------------------------------------------------
    sent_table = Table(title="Sentences and Embeddings", show_lines=True)
    sent_table.add_column("Index", justify="right")
    sent_table.add_column("Sentence")
    sent_table.add_column("Embedding (first 6 dims)")

    for i, (s, e) in enumerate(zip(sentences, embeddings)):
        sent_table.add_row(
            str(i),
            s,
            np.array2string(e[:6], precision=3, suppress_small=True)
        )

    console.print(sent_table)

    # ------------------------------------------------------------------
    # Step 4: Similarities + thresholding
    # ------------------------------------------------------------------
    sim_table = Table(
        title=f"Cosine Similarities (Threshold = {threshold})",
        show_lines=True
    )
    sim_table.add_column("Pair")
    sim_table.add_column("Similarity", justify="right")
    sim_table.add_column("Below Threshold?")
    sim_table.add_column("BREAK")

    breakpoints = []

    for i in range(len(embeddings) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i + 1])
        below = sim < threshold
        breakpoints.append(below)

        sim_table.add_row(
            f"S{i} → S{i+1}",
            f"{sim:.4f}",
            "YES" if below else "NO",
            "🔴 BREAK" if below else ""
        )

    console.print(sim_table)

    # ------------------------------------------------------------------
    # Step 5: Final chunks
    # ------------------------------------------------------------------
    chunks = []
    current_chunk = [sentences[0]]

    for i, is_break in enumerate(breakpoints):
        if is_break:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentences[i + 1]]
        else:
            current_chunk.append(sentences[i + 1])

    chunks.append(" ".join(current_chunk))

    # ------------------------------------------------------------
    # Step 6: Display final chunks
    # ------------------------------------------------------------
    for i, chunk in enumerate(chunks):
        console.print(
            Panel(
                chunk,
                title=f"Chunk {i}",
                expand=False
            )
        )



In [4]:
text = """
MEDUSA
Jelly-fish
Whole face puffed œdematous-eyes, nose, ears, lips.
Skin.––Numbness; burning, pricking heat. Vesicular eruption especially on face, arms, shoulders, and breasts. Nettlerash (Apis; Chloral; Dulc).
Female.––Marked action on lacteal glands. The secretion of milk was established after lack of it in all previous confinements.
Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep.

MEL CUM SALE
Honey with Salt
Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from ileum to ileum.
Uterine displacements, and in the commencement of metritis Sensation as if bladder were too full. Pain from sacrum towards pubes. Pain as if in ureters.
Dose.––Third to sixth potency. Honey for itching of anus and worms.

METHYLENUM COERULEUM
Aniline Dye
Methylene Blue
A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg.
Surgical kidney with large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states of apoplexy (Gisevius).
Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul smelling discharge.
A 1 per cent aqueous solution for ulcers and abscesses of cornea.
"""

naive_semantic_chunking(text, threshold=0.75)


─────────────────────────────── Level 0 — Naive Semantic Chunking (What NOT to do) ────────────────────────────────

                                             Sentences and Embeddings                                              
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Index ┃ Sentence                                                  ┃ Embedding (first 6 dims)                    ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     0 │ MEDUSA                                                    │ [ 0.008 -0.014  0.035 -0.027 -0.109  0.019] │
│       │ Jelly-fish                                                │                                             │
│       │ Whole face puffed œdematous-eyes, nose, ears, lips.       │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     1 │ Skin.––Numbness; burning, pricking heat.                  │ [ 0.019 -0.023  0.028  0.099  0.012 -0.007] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     2 │ Vesicular eruption especially on face, arms, shoulders,   │ [ 0.022 -0.024  0.022  0.033 -0.038 -0.067] │
│       │ and breasts.                                              │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     3 │ Nettlerash (Apis; Chloral; Dulc).                         │ [-0.101  0.009 -0.078 -0.098 -0.008 -0.002] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     4 │ Female.––Marked action on lacteal glands.                 │ [ 0.056 -0.135  0.017  0.019  0.039  0.049] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     5 │ The secretion of milk was established after lack of it in │ [-0.01  -0.008  0.007  0.092  0.054  0.04 ] │
│       │ all previous confinements.                                │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     6 │ Relationship.––Compare: Pyrarara, Physalia (urticaria);   │ [ 0.024 -0.049 -0.021 -0.055 -0.046  0.029] │
│       │ Urtica, Homar, Sep.                                       │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     7 │ MEL CUM SALE                                              │ [-0.028 -0.02   0.044 -0.008 -0.081  0.084] │
│       │ Honey with Salt                                           │                                             │
│       │ Prolapsus uteri and chronic metritis, especially when     │                                             │
│       │ associated with subinvolution and inflammation of the     │                                             │
│       │ cervix.                                                   │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     8 │ The special symptom leading to its selection is a feeling │ [ 0.079 -0.05  -0.003  0.091 -0.    -0.049] │
│       │ of soreness across the hypogastrium from ileum to ileum.  │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     9 │ Uterine displacements, and in the commencement of         │ [-0.012 -0.102  0.051  0.097 -0.036 -0.07 ] │
│       │ metritis Sensation as if bladder were too full.           │                                             │
├───────┼───────────────────────────────────────────────

         Cosine Similarities (Threshold = 0.75)         
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Below Threshold? ┃ BREAK    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S1 → S2   │     0.4403 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S2 → S3   │     0.0192 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S3 → S4   │     0.0852 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S4 → S5   │     0.3052 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S5 → S6   │     0.0378 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S6 → S7   │     0.2145 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S7 → S8   │     0.2816 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S8 → S9   │     0.4499 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S9 → S10  │     0.4174 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S10 → S11 │     0.5411 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S11 → S12 │     0.2616 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S12 → S13 │     0.1467 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S13 → S14 │     0.3779 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S14 → S15 │     0.1279 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S15 → S16 │     0.2231 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S16 → S17 │     0.1720 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S17 → S18 │     0.3420 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S18 → S19 │     0.4766 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S19 → S20 │     0.2057 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S20 → S21 │     0.3742 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S21 → S22 │     0.2566 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S22 → S23 │     0.2661 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S23 → S24 │     0.2048 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S24 → S25 │     0.3949 │ YES              │ 🔴 BREAK │
└───────────┴────────────┴──────────────────┴──────────┘

╭────────────────────── Chunk 0 ──────────────────────╮
│ MEDUSA                                              │
│ Jelly-fish                                          │
│ Whole face puffed œdematous-eyes, nose, ears, lips. │
╰─────────────────────────────────────────────────────╯

╭──────────────── Chunk 1 ─────────────────╮
│ Skin.––Numbness; burning, pricking heat. │
╰──────────────────────────────────────────╯

...

╭───── Chunk 21 ──────╮
│ Backache, sciatica. │
╰─────────────────────╯

╭────────────── Chunk 22 ──────────────╮
│ Later states of apoplexy (Gisevius). │
╰──────────────────────────────────────╯

╭─────── Chunk 23 ───────╮
│ Dose.––3x attenuation. │
╰────────────────────────╯

╭─────────────────────────────────── Chunk 24 ───────────────────────────────────╮
│ A 2 per cent solution locally, in chronic otitis with foul smelling discharge. │
╰────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────── Chunk 25 ─────────────────────────────╮
│ A 1 per cent aqueous solution for ulcers and abscesses of cornea. │
╰───────────────────────────────────────────────────────────────────╯

In [5]:
naive_semantic_chunking(text, threshold=0.4)


─────────────────────────────── Level 0 — Naive Semantic Chunking (What NOT to do) ────────────────────────────────

                                             Sentences and Embeddings                                              
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Index ┃ Sentence                                                  ┃ Embedding (first 6 dims)                    ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     0 │ MEDUSA                                                    │ [ 0.008 -0.014  0.035 -0.027 -0.109  0.019] │
│       │ Jelly-fish                                                │                                             │
│       │ Whole face puffed œdematous-eyes, nose, ears, lips.       │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     1 │ Skin.––Numbness; burning, pricking heat.                  │ [ 0.019 -0.023  0.028  0.099  0.012 -0.007] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     2 │ Vesicular eruption especially on face, arms, shoulders,   │ [ 0.022 -0.024  0.022  0.033 -0.038 -0.067] │
│       │ and breasts.                                              │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     3 │ Nettlerash (Apis; Chloral; Dulc).                         │ [-0.101  0.009 -0.078 -0.098 -0.008 -0.002] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     4 │ Female.––Marked action on lacteal glands.                 │ [ 0.056 -0.135  0.017  0.019  0.039  0.049] │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     5 │ The secretion of milk was established after lack of it in │ [-0.01  -0.008  0.007  0.092  0.054  0.04 ] │
│       │ all previous confinements.                                │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     6 │ Relationship.––Compare: Pyrarara, Physalia (urticaria);   │ [ 0.024 -0.049 -0.021 -0.055 -0.046  0.029] │
│       │ Urtica, Homar, Sep.                                       │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     7 │ MEL CUM SALE                                              │ [-0.028 -0.02   0.044 -0.008 -0.081  0.084] │
│       │ Honey with Salt                                           │                                             │
│       │ Prolapsus uteri and chronic metritis, especially when     │                                             │
│       │ associated with subinvolution and inflammation of the     │                                             │
│       │ cervix.                                                   │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     8 │ The special symptom leading to its selection is a feeling │ [ 0.079 -0.05  -0.003  0.091 -0.    -0.049] │
│       │ of soreness across the hypogastrium from ileum to ileum.  │                                             │
├───────┼───────────────────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     9 │ Uterine displacements, and in the commencement of         │ [-0.012 -0.102  0.051  0.097 -0.036 -0.07 ] │
│       │ metritis Sensation as if bladder were too full.           │                                             │
├───────┼───────────────────────────────────────────────

         Cosine Similarities (Threshold = 0.4)          
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Below Threshold? ┃ BREAK    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S1 → S2   │     0.4403 │ NO               │          │
├───────────┼────────────┼──────────────────┼──────────┤
│ S2 → S3   │     0.0192 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S3 → S4   │     0.0852 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S4 → S5   │     0.3052 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S5 → S6   │     0.0378 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S6 → S7   │     0.2145 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S7 → S8   │     0.2816 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S8 → S9   │     0.4499 │ NO               │          │
├───────────┼────────────┼──────────────────┼──────────┤
│ S9 → S10  │     0.4174 │ NO               │          │
├───────────┼────────────┼──────────────────┼──────────┤
│ S10 → S11 │     0.5411 │ NO               │          │
├───────────┼────────────┼──────────────────┼──────────┤
│ S11 → S12 │     0.2616 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S12 → S13 │     0.1467 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S13 → S14 │     0.3779 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S14 → S15 │     0.1279 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S15 → S16 │     0.2231 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S16 → S17 │     0.1720 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S17 → S18 │     0.3420 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S18 → S19 │     0.4766 │ NO               │          │
├───────────┼────────────┼──────────────────┼──────────┤
│ S19 → S20 │     0.2057 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S20 → S21 │     0.3742 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S21 → S22 │     0.2566 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S22 → S23 │     0.2661 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S23 → S24 │     0.2048 │ YES              │ 🔴 BREAK │
├───────────┼────────────┼──────────────────┼──────────┤
│ S24 → S25 │     0.3949 │ YES              │ 🔴 BREAK │
└───────────┴────────────┴──────────────────┴──────────┘

╭────────────────────── Chunk 0 ──────────────────────╮
│ MEDUSA                                              │
│ Jelly-fish                                          │
│ Whole face puffed œdematous-eyes, nose, ears, lips. │
╰─────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Chunk 1 ───────────────────────────────────────────────────╮
│ Skin.––Numbness; burning, pricking heat. Vesicular eruption especially on face, arms, shoulders, and breasts. │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───── Chunk 16 ──────╮
│ Backache, sciatica. │
╰─────────────────────╯

╭────────────── Chunk 17 ──────────────╮
│ Later states of apoplexy (Gisevius). │
╰──────────────────────────────────────╯

╭─────── Chunk 18 ───────╮
│ Dose.––3x attenuation. │
╰────────────────────────╯

╭─────────────────────────────────── Chunk 19 ───────────────────────────────────╮
│ A 2 per cent solution locally, in chronic otitis with foul smelling discharge. │
╰────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────── Chunk 20 ─────────────────────────────╮
│ A 1 per cent aqueous solution for ulcers and abscesses of cornea. │
╰───────────────────────────────────────────────────────────────────╯

##### Threshold and Model may or may not fit the document. Defining these, we add restrictions ourselves without understanding the document.

### Level 1: Statistical Split

In [6]:
def stddev_semantic_chunker(
    text: str,
    k: float = 1.0,
    model_name: str = "all-MiniLM-L6-v2",
):
    """
    Level 1 — Standard Deviation based Semantic Chunking
    Adaptive, but statistically fragile for uniform text.
    """

    console.rule("[bold cyan]Level 1 — Standard Deviation Semantic Chunking")

    # ------------------------------------------------------------
    # Step 1: Sentence segmentation
    # ------------------------------------------------------------
    sentences = [
        s.strip()
        for s in re.split(r"(?<=[.!?])\s+", text)
        if s.strip()
    ]

    # ------------------------------------------------------------
    # Step 2: Sentence embeddings (raw, cosine used explicitly)
    # ------------------------------------------------------------
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences)

    # ------------------------------------------------------------
    # Step 3: Compute adjacent similarities
    # ------------------------------------------------------------
    similarities = []
    for i in range(len(embeddings) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i + 1])
        similarities.append(sim)

    similarities = np.array(similarities)

    # ------------------------------------------------------------
    # Step 4: Compute statistics
    # ------------------------------------------------------------
    mu = similarities.mean()
    sigma = similarities.std()

    cutoff = mu - k * sigma

    stats_panel = Panel(
        f"""
Mean (μ): {mu:.4f}
Std-dev (σ): {sigma:.4f}
k: {k}
Cutoff (μ - kσ): {cutoff:.4f}
""".strip(),
        title="Similarity Statistics",
        expand=False
    )

    console.print(stats_panel)

    # ------------------------------------------------------------
    # Step 5: Similarity + breakpoint table
    # ------------------------------------------------------------
    sim_table = Table(
        title="Adjacent Similarities with Std-dev Rule",
        show_lines=True
    )
    sim_table.add_column("Pair")
    sim_table.add_column("Similarity", justify="right")
    sim_table.add_column("μ - kσ", justify="right")
    sim_table.add_column("Decision")
    sim_table.add_column("Reason")

    breakpoints = []

    for i, sim in enumerate(similarities):
        do_break = sim < cutoff
        breakpoints.append(do_break)

        sim_table.add_row(
            f"S{i} → S{i+1}",
            f"{sim:.4f}",
            f"{cutoff:.4f}",
            "BREAK" if do_break else "KEEP",
            "statistical drop" if do_break else "within variance"
        )

    console.print(sim_table)

    # ------------------------------------------------------------
    # Step 6: Chunk assembly
    # ------------------------------------------------------------
    chunks = []
    current = [sentences[0]]

    for i, brk in enumerate(breakpoints):
        if brk:
            chunks.append(" ".join(current))
            current = [sentences[i + 1]]
        else:
            current.append(sentences[i + 1])

    chunks.append(" ".join(current))

    # ------------------------------------------------------------
    # Step 7: Display chunks
    # ------------------------------------------------------------
    for i, chunk in enumerate(chunks):
        console.print(
            Panel(
                chunk,
                title=f"Chunk {i}",
                expand=False
            )
        )

In [7]:
stddev_semantic_chunker(text)

───────────────────────────────── Level 1 — Standard Deviation Semantic Chunking ──────────────────────────────────

╭─ Similarity Statistics ─╮
│ Mean (μ): 0.2778        │
│ Std-dev (σ): 0.1349     │
│ k: 1.0                  │
│ Cutoff (μ - kσ): 0.1429 │
╰─────────────────────────╯

             Adjacent Similarities with Std-dev Rule             
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ μ - kσ ┃ Decision ┃ Reason           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S1 → S2   │     0.4403 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S2 → S3   │     0.0192 │ 0.1429 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S3 → S4   │     0.0852 │ 0.1429 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S4 → S5   │     0.3052 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S5 → S6   │     0.0378 │ 0.1429 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S6 → S7   │     0.2145 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S7 → S8   │     0.2816 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S8 → S9   │     0.4499 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S9 → S10  │     0.4174 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S10 → S11 │     0.5411 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S11 → S12 │     0.2616 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S12 → S13 │     0.1467 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S13 → S14 │     0.3779 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S14 → S15 │     0.1279 │ 0.1429 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S15 → S16 │     0.2231 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S16 → S17 │     0.1720 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S17 → S18 │     0.3420 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S18 → S19 │     0.4766 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S19 → S20 │     0.2057 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S20 → S21 │     0.3742 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S21 → S22 │     0.2566 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S22 → S23 │     0.2661 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S23 → S24 │     0.2048 │ 0.1429 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S24 → S25 │     0.3949 │ 0.1429 │ KEEP     │ within variance  │
└───────────┴────────────┴────────┴──────────┴──────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────── Chunk 1 ─────────────╮
│ Nettlerash (Apis; Chloral; Dulc). │
╰───────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Female.––Marked action on lacteal glands. The secretion of milk was established after lack of it in all         │
│ previous confinements.                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection.                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 4 ────────────────────────────────────────────────────╮
│ Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal nephritis. Urine         │
│ acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical kidney with      │
│ large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states of apoplexy  │
│ (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul smelling          │
│ discharge. A 1 per cent aqueous solution for ulcers and abscesses of cornea.                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### Breakpoints should be relative to the document, not absolute numbers.

In [8]:
stddev_semantic_chunker(text=text,k=0.5)

───────────────────────────────── Level 1 — Standard Deviation Semantic Chunking ──────────────────────────────────

╭─ Similarity Statistics ─╮
│ Mean (μ): 0.2778        │
│ Std-dev (σ): 0.1349     │
│ k: 0.5                  │
│ Cutoff (μ - kσ): 0.2103 │
╰─────────────────────────╯

             Adjacent Similarities with Std-dev Rule             
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ μ - kσ ┃ Decision ┃ Reason           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S1 → S2   │     0.4403 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S2 → S3   │     0.0192 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S3 → S4   │     0.0852 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S4 → S5   │     0.3052 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S5 → S6   │     0.0378 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S6 → S7   │     0.2145 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S7 → S8   │     0.2816 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S8 → S9   │     0.4499 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S9 → S10  │     0.4174 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S10 → S11 │     0.5411 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S11 → S12 │     0.2616 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S12 → S13 │     0.1467 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S13 → S14 │     0.3779 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S14 → S15 │     0.1279 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S15 → S16 │     0.2231 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S16 → S17 │     0.1720 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S17 → S18 │     0.3420 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S18 → S19 │     0.4766 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S19 → S20 │     0.2057 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S20 → S21 │     0.3742 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S21 → S22 │     0.2566 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S22 → S23 │     0.2661 │ 0.2103 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S23 → S24 │     0.2048 │ 0.2103 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S24 → S25 │     0.3949 │ 0.2103 │ KEEP     │ within variance  │
└───────────┴────────────┴────────┴──────────┴──────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────── Chunk 1 ─────────────╮
│ Nettlerash (Apis; Chloral; Dulc). │
╰───────────────────────────────────╯

...

╭──────────────────────────────────────────────────── Chunk 4 ────────────────────────────────────────────────────╮
│ Honey for itching of anus and worms. METHYLENUM COERULEUM                                                       │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection.                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Chunk 5 ─────────────────────────────────────────────╮
│ Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal nephritis. │
╰───────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 6 ────────────────────────────────────────────────────╮
│ Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical kidney     │
│ with large amount of pus in urine.                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 7 ────────────────────────────────────────────────────╮
│ Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states of apoplexy (Gisevius). Dose.––3x          │
│ attenuation.                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 8 ────────────────────────────────────────────────────╮
│ A 2 per cent solution locally, in chronic otitis with foul smelling discharge. A 1 per cent aqueous solution    │
│ for ulcers and abscesses of cornea.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
stddev_semantic_chunker(text=text,k=1.5)

───────────────────────────────── Level 1 — Standard Deviation Semantic Chunking ──────────────────────────────────

╭─ Similarity Statistics ─╮
│ Mean (μ): 0.2778        │
│ Std-dev (σ): 0.1349     │
│ k: 1.5                  │
│ Cutoff (μ - kσ): 0.0754 │
╰─────────────────────────╯

             Adjacent Similarities with Std-dev Rule             
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ μ - kσ ┃ Decision ┃ Reason           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S1 → S2   │     0.4403 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S2 → S3   │     0.0192 │ 0.0754 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S3 → S4   │     0.0852 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S4 → S5   │     0.3052 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S5 → S6   │     0.0378 │ 0.0754 │ BREAK    │ statistical drop │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S6 → S7   │     0.2145 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S7 → S8   │     0.2816 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S8 → S9   │     0.4499 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S9 → S10  │     0.4174 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S10 → S11 │     0.5411 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S11 → S12 │     0.2616 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S12 → S13 │     0.1467 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S13 → S14 │     0.3779 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S14 → S15 │     0.1279 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S15 → S16 │     0.2231 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S16 → S17 │     0.1720 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S17 → S18 │     0.3420 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S18 → S19 │     0.4766 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S19 → S20 │     0.2057 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S20 → S21 │     0.3742 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S21 → S22 │     0.2566 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S22 → S23 │     0.2661 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S23 → S24 │     0.2048 │ 0.0754 │ KEEP     │ within variance  │
├───────────┼────────────┼────────┼──────────┼──────────────────┤
│ S24 → S25 │     0.3949 │ 0.0754 │ KEEP     │ within variance  │
└───────────┴────────────┴────────┴──────────┴──────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 1 ────────────────────────────────────────────────────╮
│ Nettlerash (Apis; Chloral; Dulc). Female.––Marked action on lacteal glands. The secretion of milk was           │
│ established after lack of it in all previous confinements.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states  │
│ of apoplexy (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul       │
│ smelling discharge. A 1 per cent aqueous solution for ulcers and abscesses of cornea.                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### k defines our choice of normal, aggressive, conservative chunking

### Percentile Based Semantic Chunking

In [10]:
def percentile_semantic_chunker(
    text: str,
    percentile: float = 90,
    model_name: str = "all-MiniLM-L6-v2",
):
    """
    Level 2 — Percentile-based Semantic Chunking
    No normalization. Cos(theta) computed explicitly.
    """

    console.rule("[bold green]Level 2 — Percentile Semantic Chunking")

    # ------------------------------------------------------------
    # Step 1: Sentence segmentation
    # ------------------------------------------------------------
    sentences = [
        s.strip()
        for s in re.split(r"(?<=[.!?])\s+", text)
        if s.strip()
    ]

    # ------------------------------------------------------------
    # Step 2: Embeddings (raw)
    # ------------------------------------------------------------
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences)

    # ------------------------------------------------------------
    # Step 3: Adjacent cosine similarities
    # ------------------------------------------------------------
    similarities = []
    for i in range(len(embeddings) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i + 1])
        similarities.append(sim)

    similarities = np.array(similarities)

    # ------------------------------------------------------------
    # Step 4: Percentile threshold
    # ------------------------------------------------------------
    cutoff = np.percentile(similarities, 100 - percentile)

    stats_panel = Panel(
        f"""
Percentile: {percentile}
Cutoff similarity (bottom {100 - percentile}%):
{cutoff:.4f}
""".strip(),
        title="Percentile Statistics",
        expand=False
    )
    console.print(stats_panel)

    # ------------------------------------------------------------
    # Step 5: Similarity + breakpoint table
    # ------------------------------------------------------------
    sim_table = Table(
        title="Adjacent Similarities (Percentile Rule)",
        show_lines=True
    )
    sim_table.add_column("Pair")
    sim_table.add_column("Similarity", justify="right")
    sim_table.add_column("Cutoff", justify="right")
    sim_table.add_column("Decision")
    sim_table.add_column("Reason")

    breakpoints = []

    for i, sim in enumerate(similarities):
        do_break = sim <= cutoff
        breakpoints.append(do_break)

        sim_table.add_row(
            f"S{i} → S{i+1}",
            f"{sim:.4f}",
            f"{cutoff:.4f}",
            "BREAK" if do_break else "KEEP",
            "lowest similarity bucket" if do_break else "above percentile"
        )

    console.print(sim_table)

    # ------------------------------------------------------------
    # Step 6: Chunk assembly
    # ------------------------------------------------------------
    chunks = []
    current = [sentences[0]]

    for i, brk in enumerate(breakpoints):
        if brk:
            chunks.append(" ".join(current))
            current = [sentences[i + 1]]
        else:
            current.append(sentences[i + 1])

    chunks.append(" ".join(current))

    # ------------------------------------------------------------
    # Step 7: Display final chunks
    # ------------------------------------------------------------
    for i, chunk in enumerate(chunks):
        console.print(
            Panel(
                chunk,
                title=f"Chunk {i}",
                expand=False
            )
        )

In [11]:
percentile_semantic_chunker(text=text)

───────────────────────────────────── Level 2 — Percentile Semantic Chunking ──────────────────────────────────────

╭───── Percentile Statistics ─────╮
│ Percentile: 90                  │
│ Cutoff similarity (bottom 10%): │
│ 0.1023                          │
╰─────────────────────────────────╯

                 Adjacent Similarities (Percentile Rule)                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃ Reason                   ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S1 → S2   │     0.4403 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S2 → S3   │     0.0192 │ 0.1023 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S3 → S4   │     0.0852 │ 0.1023 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S4 → S5   │     0.3052 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S5 → S6   │     0.0378 │ 0.1023 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S6 → S7   │     0.2145 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S7 → S8   │     0.2816 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S8 → S9   │     0.4499 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S9 → S10  │     0.4174 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S10 → S11 │     0.5411 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S11 → S12 │     0.2616 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S12 → S13 │     0.1467 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S13 → S14 │     0.3779 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S14 → S15 │     0.1279 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S15 → S16 │     0.2231 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S16 → S17 │     0.1720 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S17 → S18 │     0.3420 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S18 → S19 │     0.4766 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S19 → S20 │     0.2057 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S20 → S21 │     0.3742 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S21 → S22 │     0.2566 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S22 → S23 │     0.2661 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S23 → S24 │     0.2048 │ 0.1023 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S24 → S25 │     0.3949 │ 0.1023 │ KEEP     │ above percentile         │
└───────────┴────────────┴────────┴──────────┴──────────────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────── Chunk 1 ─────────────╮
│ Nettlerash (Apis; Chloral; Dulc). │
╰───────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Female.––Marked action on lacteal glands. The secretion of milk was established after lack of it in all         │
│ previous confinements.                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states  │
│ of apoplexy (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul       │
│ smelling discharge. A 1 per cent aqueous solution for ulcers and abscesses of cornea.                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
percentile_semantic_chunker(text=text,percentile=95)

───────────────────────────────────── Level 2 — Percentile Semantic Chunking ──────────────────────────────────────

╭──── Percentile Statistics ─────╮
│ Percentile: 95                 │
│ Cutoff similarity (bottom 5%): │
│ 0.0473                         │
╰────────────────────────────────╯

                 Adjacent Similarities (Percentile Rule)                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃ Reason                   ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S1 → S2   │     0.4403 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S2 → S3   │     0.0192 │ 0.0473 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S3 → S4   │     0.0852 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S4 → S5   │     0.3052 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S5 → S6   │     0.0378 │ 0.0473 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S6 → S7   │     0.2145 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S7 → S8   │     0.2816 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S8 → S9   │     0.4499 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S9 → S10  │     0.4174 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S10 → S11 │     0.5411 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S11 → S12 │     0.2616 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S12 → S13 │     0.1467 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S13 → S14 │     0.3779 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S14 → S15 │     0.1279 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S15 → S16 │     0.2231 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S16 → S17 │     0.1720 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S17 → S18 │     0.3420 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S18 → S19 │     0.4766 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S19 → S20 │     0.2057 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S20 → S21 │     0.3742 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S21 → S22 │     0.2566 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S22 → S23 │     0.2661 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S23 → S24 │     0.2048 │ 0.0473 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S24 → S25 │     0.3949 │ 0.0473 │ KEEP     │ above percentile         │
└───────────┴────────────┴────────┴──────────┴──────────────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 1 ────────────────────────────────────────────────────╮
│ Nettlerash (Apis; Chloral; Dulc). Female.––Marked action on lacteal glands. The secretion of milk was           │
│ established after lack of it in all previous confinements.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states  │
│ of apoplexy (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul       │
│ smelling discharge. A 1 per cent aqueous solution for ulcers and abscesses of cornea.                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
percentile_semantic_chunker(text=text,percentile=85)

───────────────────────────────────── Level 2 — Percentile Semantic Chunking ──────────────────────────────────────

╭───── Percentile Statistics ─────╮
│ Percentile: 85                  │
│ Cutoff similarity (bottom 15%): │
│ 0.1392                          │
╰─────────────────────────────────╯

                 Adjacent Similarities (Percentile Rule)                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃ Reason                   ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ S0 → S1   │     0.3228 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S1 → S2   │     0.4403 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S2 → S3   │     0.0192 │ 0.1392 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S3 → S4   │     0.0852 │ 0.1392 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S4 → S5   │     0.3052 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S5 → S6   │     0.0378 │ 0.1392 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S6 → S7   │     0.2145 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S7 → S8   │     0.2816 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S8 → S9   │     0.4499 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S9 → S10  │     0.4174 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S10 → S11 │     0.5411 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S11 → S12 │     0.2616 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S12 → S13 │     0.1467 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S13 → S14 │     0.3779 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S14 → S15 │     0.1279 │ 0.1392 │ BREAK    │ lowest similarity bucket │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S15 → S16 │     0.2231 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S16 → S17 │     0.1720 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S17 → S18 │     0.3420 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S18 → S19 │     0.4766 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S19 → S20 │     0.2057 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S20 → S21 │     0.3742 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S21 → S22 │     0.2566 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S22 → S23 │     0.2661 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S23 → S24 │     0.2048 │ 0.1392 │ KEEP     │ above percentile         │
├───────────┼────────────┼────────┼──────────┼──────────────────────────┤
│ S24 → S25 │     0.3949 │ 0.1392 │ KEEP     │ above percentile         │
└───────────┴────────────┴────────┴──────────┴──────────────────────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────── Chunk 1 ─────────────╮
│ Nettlerash (Apis; Chloral; Dulc). │
╰───────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Female.––Marked action on lacteal glands. The secretion of milk was established after lack of it in all         │
│ previous confinements.                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection.                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 4 ────────────────────────────────────────────────────╮
│ Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal nephritis. Urine         │
│ acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical kidney with      │
│ large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica. Later states of apoplexy  │
│ (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul smelling          │
│ discharge. A 1 per cent aqueous solution for ulcers and abscesses of cornea.                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### We still treat each sentence as an embedding

### Sentence Windows

In [14]:
def build_window(sentences, center, w):
    start = max(0, center - w)
    end = min(len(sentences), center + w + 1)
    return " ".join(sentences[start:end])

In [15]:
def windowed_semantic_chunker(
    text: str,
    window_size: int = 1,
    percentile: float = 90,
    model_name: str = "all-MiniLM-L6-v2",
):
    console.rule("[bold magenta]Level 3 — Windowed Semantic Chunking (Visible)")

    # ------------------------------------------------------------
    # Step 1: Sentence split
    # ------------------------------------------------------------
    sentences = [
        s.strip()
        for s in re.split(r"(?<=[.!?])\s+", text)
        if s.strip()
    ]

    # ------------------------------------------------------------
    # Step 2: Build windows (EXPLICIT)
    # ------------------------------------------------------------
    windows = []
    for i in range(len(sentences)):
        win_sents = build_window(sentences, i, window_size)
        windows.append(" ".join(win_sents))

    # ------------------------------------------------------------
    # Step 3: Show windows
    # ------------------------------------------------------------
    win_table = Table(
        title="Constructed Sentence Windows",
        show_lines=True
    )
    win_table.add_column("Window")
    win_table.add_column("Contains sentences")
    win_table.add_column("Window text")

    for i, win in enumerate(windows):
        indices = list(
            range(
                max(0, i - window_size),
                min(len(sentences), i + window_size + 1)
            )
        )
        win_table.add_row(
            f"W{i}",
            ", ".join(f"S{j}" for j in indices),
            win
        )

    console.print(win_table)

    # ------------------------------------------------------------
    # Step 4: Embed windows
    # ------------------------------------------------------------
    model = SentenceTransformer(model_name)
    embeddings = model.encode(windows)

    # ------------------------------------------------------------
    # Step 5: Window-to-window similarities
    # ------------------------------------------------------------
    similarities = []
    for i in range(len(embeddings) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i + 1])
        similarities.append(sim)

    similarities = np.array(similarities)

    cutoff = np.percentile(similarities, 100 - percentile)

    # ------------------------------------------------------------
    # Step 6: Similarity diagnostics
    # ------------------------------------------------------------
    sim_table = Table(
        title="Window Similarities & Breakpoints",
        show_lines=True
    )
    sim_table.add_column("Pair")
    sim_table.add_column("Similarity", justify="right")
    sim_table.add_column("Cutoff", justify="right")
    sim_table.add_column("Decision")

    breakpoints = []

    for i, sim in enumerate(similarities):
        do_break = sim <= cutoff
        breakpoints.append(do_break)

        sim_table.add_row(
            f"W{i} → W{i+1}",
            f"{sim:.4f}",
            f"{cutoff:.4f}",
            "BREAK" if do_break else "KEEP"
        )

    console.print(sim_table)

    # ------------------------------------------------------------
    # Step 7: Chunk assembly (sentence aligned)
    # ------------------------------------------------------------
    chunks = []
    current = [sentences[0]]

    for i, brk in enumerate(breakpoints):
        if brk:
            chunks.append(" ".join(current))
            current = [sentences[i + 1]]
        else:
            current.append(sentences[i + 1])

    chunks.append(" ".join(current))

    # ------------------------------------------------------------
    # Step 8: Display chunks
    # ------------------------------------------------------------
    for i, chunk in enumerate(chunks):
        console.print(
            Panel(
                chunk,
                title=f"Chunk {i}",
                expand=False
            )
        )



In [16]:
windowed_semantic_chunker(text=text,window_size=1)

───────────────────────────────── Level 3 — Windowed Semantic Chunking (Visible) ──────────────────────────────────

                                           Constructed Sentence Windows                                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Window ┃ Contains sentences ┃ Window text                                                                       ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ W0     │ S0, S1             │ M E D U S A                                                                       │
│        │                    │  J e l l y - f i s h                                                              │
│        │                    │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s ,   n o s e ,    │
│        │                    │ e a r s ,   l i p s .   S k i n . – – N u m b n e s s ;   b u r n i n g ,   p r i │
│        │                    │ c k i n g   h e a t .                                                             │
├────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ W1     │ S0, S1, S2         │ M E D U S A                                                                       │
│        │                    │  J e l l y - f i s h                                                              │
│        │                    │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s ,   n o s e ,    │
│        │                    │ e a r s ,   l i p s .   S k i n . – – N u m b n e s s ;   b u r n i n g ,   p r i │
│        │                    │ c k i n g   h e a t .   V e s i c u l a r   e r u p t i o n   e s p e c i a l l y │
│        │                    │ o n   f a c e ,   a r m s ,   s h o u l d e r s ,   a n d   b r e a s t s .       │
├────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ W2     │ S1, S2, S3         │ S k i n . – – N u m b n e s s ;   b u r n i n g ,   p r i c k i n g   h e a t .   │
│        │                    │ V e s i c u l a r   e r u p t i o n   e s p e c i a l l y   o n   f a c e ,   a r │
│        │                    │ m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t l e r a s h   ( A │
│        │                    │ p i s ;   C h l o r a l ;   D u l c ) .                                           │
├────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ W3     │ S2, S3, S4         │ V e s i c u l a r   e r u p t i o n   e s p e c i a l l y   o n   f a c e ,   a r │
│        │                    │ m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t l e r a s h   ( A │
│        │                    │ p i s ;   C h l o r a l ;   D u l c ) .   F e m a l e . – – M a r k e d   a c t i │
│        │                    │ o n   o n   l a c t e a l   g l a n d s .                                         │
├────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ W4     │ S3, S4, S5         │ N e t t l e r a s h   ( A p i s ;   C h l o r a l ;   D u l c ) .   F e m a l e . │
│        │                    │ – – M a r k e d   a c t i o n   o n   l a c t e a l   g l a n d s .   T h e   s e │
│        │                    │ c r e t i o n   o f   m i l k   w a s   e s t a b l i s h e d   a f t e r   l a c │
│        │                    │ k   o f   i t   i n   a l l   p r e v i o u s   c o n f i n e m e n t s .         │
├────────┼────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ W5     │ S4, S5, S6         │ F e m a l e . – – M a r k e d   a c t i o n   o n   l a c t e a l   g l a n d s . │
│        │                    │ T h e   s e c r e t i o n   o f   m i l k   w a s   e s t a b l i s h e d   a f t │
│        │                    │ e r   l a c k   o f   i 

      Window Similarities & Breakpoints       
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ W0 → W1   │     0.9585 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W1 → W2   │     0.9066 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W2 → W3   │     0.9521 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W3 → W4   │     0.9648 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W4 → W5   │     0.9378 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W5 → W6   │     0.8718 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W6 → W7   │     0.9894 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W7 → W8   │     0.9774 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W8 → W9   │     0.9688 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W9 → W10  │     0.9404 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W10 → W11 │     0.9087 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W11 → W12 │     0.9666 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W12 → W13 │     0.8589 │ 0.8640 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W13 → W14 │     0.9800 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W14 → W15 │     0.9776 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W15 → W16 │     0.8786 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W16 → W17 │     0.9699 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W17 → W18 │     0.9485 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W18 → W19 │     0.8881 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W19 → W20 │     0.9239 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W20 → W21 │     0.9052 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W21 → W22 │     0.7106 │ 0.8640 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W22 → W23 │     0.9338 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W23 → W24 │     0.8751 │ 0.8640 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W24 → W25 │     0.8483 │ 0.8640 │ BREAK    │
└───────────┴────────────┴────────┴──────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts. Nettlerash (Apis; Chloral; Dulc). Female.––Marked action on   │
│ lacteal glands. The secretion of milk was established after lack of it in all previous confinements.            │
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency.                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 1 ────────────────────────────────────────────────────╮
│ Honey for itching of anus and worms. METHYLENUM COERULEUM                                                       │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine. Gonorrhœal rheumatism and cystitis. Backache, sciatica.               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Later states of apoplexy (Gisevius). Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis    │
│ with foul smelling discharge.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── Chunk 3 ─────────────────────────────╮
│ A 1 per cent aqueous solution for ulcers and abscesses of cornea. │
╰───────────────────────────────────────────────────────────────────╯

In [17]:
windowed_semantic_chunker(text=text,window_size=2)

───────────────────────────────── Level 3 — Windowed Semantic Chunking (Visible) ──────────────────────────────────

                                           Constructed Sentence Windows                                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Window ┃ Contains sentences      ┃ Window text                                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ W0     │ S0, S1, S2              │ M E D U S A                                                                  │
│        │                         │  J e l l y - f i s h                                                         │
│        │                         │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s ,   n o s e │
│        │                         │ ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s ;   b u r n i n g  │
│        │                         │ ,   p r i c k i n g   h e a t .   V e s i c u l a r   e r u p t i o n   e s  │
│        │                         │ p e c i a l l y   o n   f a c e ,   a r m s ,   s h o u l d e r s ,   a n d  │
│        │                         │ b r e a s t s .                                                              │
├────────┼─────────────────────────┼──────────────────────────────────────────────────────────────────────────────┤
│ W1     │ S0, S1, S2, S3          │ M E D U S A                                                                  │
│        │                         │  J e l l y - f i s h                                                         │
│        │                         │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s ,   n o s e │
│        │                         │ ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s ;   b u r n i n g  │
│        │                         │ ,   p r i c k i n g   h e a t .   V e s i c u l a r   e r u p t i o n   e s  │
│        │                         │ p e c i a l l y   o n   f a c e ,   a r m s ,   s h o u l d e r s ,   a n d  │
│        │                         │ b r e a s t s .   N e t t l e r a s h   ( A p i s ;   C h l o r a l ;   D u  │
│        │                         │ l c ) .                                                                      │
├────────┼─────────────────────────┼──────────────────────────────────────────────────────────────────────────────┤
│ W2     │ S0, S1, S2, S3, S4      │ M E D U S A                                                                  │
│        │                         │  J e l l y - f i s h                                                         │
│        │                         │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s ,   n o s e │
│        │                         │ ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s ;   b u r n i n g  │
│        │                         │ ,   p r i c k i n g   h e a t .   V e s i c u l a r   e r u p t i o n   e s  │
│        │                         │ p e c i a l l y   o n   f a c e ,   a r m s ,   s h o u l d e r s ,   a n d  │
│        │                         │ b r e a s t s .   N e t t l e r a s h   ( A p i s ;   C h l o r a l ;   D u  │
│        │                         │ l c ) .   F e m a l e . – – M a r k e d   a c t i o n   o n   l a c t e a l  │
│        │                         │ g l a n d s .                                                                │
├────────┼─────────────────────────┼──────────────────────────────────────────────────────────────────────────────┤
│ W3     │ S1, S2, S3, S4, S5      │ S k i n . – – N u m b n e s s ;   b u r n i n g ,   p r i c k i n g   h e a  │
│        │                         │ t .   V e s i c u l a r   e r u p t i o n   e s p e c i a l l y   o n   f a  │
│        │                         │ c e ,   a r m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t  │
│        │                         │ l e r a s h   ( A p

      Window Similarities & Breakpoints       
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ W0 → W1   │     0.9621 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W1 → W2   │     0.9845 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W2 → W3   │     0.9580 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W3 → W4   │     0.9717 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W4 → W5   │     0.9891 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W5 → W6   │     0.9875 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W6 → W7   │     0.9757 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W7 → W8   │     0.9894 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W8 → W9   │     0.9774 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W9 → W10  │     0.9790 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W10 → W11 │     0.9635 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W11 → W12 │     0.9503 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W12 → W13 │     0.9949 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W13 → W14 │     0.9887 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W14 → W15 │     0.9865 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W15 → W16 │     0.9846 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W16 → W17 │     0.9708 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W17 → W18 │     0.9657 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W18 → W19 │     0.9690 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W19 → W20 │     0.9196 │ 0.9473 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W20 → W21 │     0.8395 │ 0.9473 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W21 → W22 │     0.9489 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W22 → W23 │     0.9462 │ 0.9473 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W23 → W24 │     0.9835 │ 0.9473 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W24 → W25 │     0.9498 │ 0.9473 │ KEEP     │
└───────────┴────────────┴────────┴──────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts. Nettlerash (Apis; Chloral; Dulc). Female.––Marked action on   │
│ lacteal glands. The secretion of milk was established after lack of it in all previous confinements.            │
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine.                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────── Chunk 1 ──────────────╮
│ Gonorrhœal rheumatism and cystitis. │
╰─────────────────────────────────────╯

╭──────────────────────── Chunk 2 ─────────────────────────╮
│ Backache, sciatica. Later states of apoplexy (Gisevius). │
╰──────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ Dose.––3x attenuation. A 2 per cent solution locally, in chronic otitis with foul smelling discharge. A 1 per   │
│ cent aqueous solution for ulcers and abscesses of cornea.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
windowed_semantic_chunker(text=text,window_size=3)

───────────────────────────────── Level 3 — Windowed Semantic Chunking (Visible) ──────────────────────────────────

                                           Constructed Sentence Windows                                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Window ┃ Contains sentences                ┃ Window text                                                        ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ W0     │ S0, S1, S2, S3                    │ M E D U S A                                                        │
│        │                                   │  J e l l y - f i s h                                               │
│        │                                   │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s , │
│        │                                   │ n o s e ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s  │
│        │                                   │ ;   b u r n i n g ,   p r i c k i n g   h e a t .   V e s i c u l  │
│        │                                   │ a r   e r u p t i o n   e s p e c i a l l y   o n   f a c e ,   a  │
│        │                                   │ r m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t  │
│        │                                   │ l e r a s h   ( A p i s ;   C h l o r a l ;   D u l c ) .          │
├────────┼───────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ W1     │ S0, S1, S2, S3, S4                │ M E D U S A                                                        │
│        │                                   │  J e l l y - f i s h                                               │
│        │                                   │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s , │
│        │                                   │ n o s e ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s  │
│        │                                   │ ;   b u r n i n g ,   p r i c k i n g   h e a t .   V e s i c u l  │
│        │                                   │ a r   e r u p t i o n   e s p e c i a l l y   o n   f a c e ,   a  │
│        │                                   │ r m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t  │
│        │                                   │ l e r a s h   ( A p i s ;   C h l o r a l ;   D u l c ) .   F e m  │
│        │                                   │ a l e . – – M a r k e d   a c t i o n   o n   l a c t e a l   g l  │
│        │                                   │ a n d s .                                                          │
├────────┼───────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ W2     │ S0, S1, S2, S3, S4, S5            │ M E D U S A                                                        │
│        │                                   │  J e l l y - f i s h                                               │
│        │                                   │  W h o l e   f a c e   p u f f e d   œ d e m a t o u s - e y e s , │
│        │                                   │ n o s e ,   e a r s ,   l i p s .   S k i n . – – N u m b n e s s  │
│        │                                   │ ;   b u r n i n g ,   p r i c k i n g   h e a t .   V e s i c u l  │
│        │                                   │ a r   e r u p t i o n   e s p e c i a l l y   o n   f a c e ,   a  │
│        │                                   │ r m s ,   s h o u l d e r s ,   a n d   b r e a s t s .   N e t t  │
│        │                                   │ l e r a s h   ( A p i s ;   C h l o r a l ;   D u l c ) .   F e m  │
│        │                                   │ a l e . – – M a r k e d   a c t i o n   o n   l a c t e a l   g l  │
│        │                                   │ a n d s .   T h e   s e c r e t i o n   o f   m i l k   w a s   e  │
│        │                                   │ s t a b l

      Window Similarities & Breakpoints       
┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Pair      ┃ Similarity ┃ Cutoff ┃ Decision ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ W0 → W1   │     0.9845 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W1 → W2   │     0.9881 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W2 → W3   │     1.0000 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W3 → W4   │     0.9763 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W4 → W5   │     0.9761 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W5 → W6   │     0.9891 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W6 → W7   │     0.9875 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W7 → W8   │     0.9757 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W8 → W9   │     0.9894 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W9 → W10  │     0.9774 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W10 → W11 │     0.9806 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W11 → W12 │     0.9705 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W12 → W13 │     0.9840 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W13 → W14 │     0.9949 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W14 → W15 │     0.9887 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W15 → W16 │     0.9865 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W16 → W17 │     0.9846 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W17 → W18 │     0.9751 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W18 → W19 │     0.9700 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W19 → W20 │     0.9307 │ 0.9642 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W20 → W21 │     0.9457 │ 0.9642 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W21 → W22 │     0.9683 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W22 → W23 │     0.9786 │ 0.9642 │ KEEP     │
├───────────┼────────────┼────────┼──────────┤
│ W23 → W24 │     0.9616 │ 0.9642 │ BREAK    │
├───────────┼────────────┼────────┼──────────┤
│ W24 → W25 │     0.9835 │ 0.9642 │ KEEP     │
└───────────┴────────────┴────────┴──────────┘

╭──────────────────────────────────────────────────── Chunk 0 ────────────────────────────────────────────────────╮
│ MEDUSA                                                                                                          │
│ Jelly-fish                                                                                                      │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. Vesicular eruption │
│ especially on face, arms, shoulders, and breasts. Nettlerash (Apis; Chloral; Dulc). Female.––Marked action on   │
│ lacteal glands. The secretion of milk was established after lack of it in all previous confinements.            │
│ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE                        │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum. Uterine displacements, and in the commencement of metritis Sensation as if bladder were too     │
│ full. Pain from sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching   │
│ of anus and worms. METHYLENUM COERULEUM                                                                         │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy. Nephritis (acute parenchymatous), scarlatinal           │
│ nephritis. Urine acquires a green color. Bladder irritation from its use antidoted by a little nutmeg. Surgical │
│ kidney with large amount of pus in urine.                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────── Chunk 1 ──────────────╮
│ Gonorrhœal rheumatism and cystitis. │
╰─────────────────────────────────────╯

╭──────────────────────────────────── Chunk 2 ────────────────────────────────────╮
│ Backache, sciatica. Later states of apoplexy (Gisevius). Dose.––3x attenuation. │
╰─────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ A 2 per cent solution locally, in chronic otitis with foul smelling discharge. A 1 per cent aqueous solution    │
│ for ulcers and abscesses of cornea.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
def guarded_semantic_chunker(
    text: str,
    window_size: int = 1,
    percentile: float = 90,
    min_drop: float = 0.05,
    min_sentences: int = 2,
    model_name: str = "all-MiniLM-L6-v2",
):
    console.rule("[bold blue]Level 4 — Guarded Semantic Chunking (Verbose & Fixed)")

    # -------------------------------
    # Step 1: Sentence split
    # -------------------------------
    sentences = [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]
    n_sent = len(sentences)

    # -------------------------------
    # Step 2: Build windows
    # -------------------------------
    windows = []
    window_texts = []
    for i in range(n_sent):
        start = max(0, i - window_size)
        end = min(n_sent, i + window_size + 1)
        win_sentences = sentences[start:end]
        windows.append(win_sentences)
        window_texts.append(" ".join(win_sentences))  # for embedding

    # -------------------------------
    # Step 3: Show windows
    # -------------------------------
    win_table = Table(title="Windows", show_lines=True)
    win_table.add_column("W#")
    win_table.add_column("Sentences in Window")

    for i, win in enumerate(windows):
        # Just show sentence texts; indices optional
        win_table.add_row(f"W{i}", " | ".join(win))

    console.print(win_table)

    # -------------------------------
    # Step 4: Embed windows
    # -------------------------------
    model = SentenceTransformer(model_name)
    embeddings = model.encode(window_texts)

    # -------------------------------
    # Step 5: Compute similarities
    # -------------------------------
    similarities = []
    for i in range(len(embeddings) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i + 1])
        similarities.append(sim)
    similarities = np.array(similarities)

    cutoff = np.percentile(similarities, 100 - percentile)

    # -------------------------------
    # Step 6: Verbose reasoning table
    # -------------------------------
    table = Table(title="Split Reasoning per Window Pair", show_lines=True)
    table.add_column("Window Pair")
    table.add_column("Similarity", justify="right")
    table.add_column("Cutoff", justify="right")
    table.add_column("Drop from prev", justify="right")
    table.add_column("Decision")
    table.add_column("Reasoning")

    breakpoints = []
    prev_sim = similarities[0] if len(similarities) > 0 else 1.0  # guard first
    for i, sim in enumerate(similarities):
        drop = prev_sim - sim
        do_break = (sim <= cutoff) and (drop >= min_drop) and (len(windows[i]) >= min_sentences)
        reason = []

        if sim <= cutoff:
            reason.append("below percentile cutoff")
        if drop >= min_drop:
            reason.append("significant drop")
        if len(windows[i]) < min_sentences:
            reason.append("window too short")

        breakpoints.append(do_break)
        prev_sim = sim

        table.add_row(
            f"W{i} → W{i+1}",
            f"{sim:.4f}",
            f"{cutoff:.4f}",
            f"{drop:.4f}",
            "BREAK" if do_break else "KEEP",
            ", ".join(reason) if reason else "OK"
        )

    console.print(table)

    # -------------------------------
    # Step 7: Chunk assembly
    # -------------------------------
    chunks = []
    current = [sentences[0]] if n_sent > 0 else []

    for i, brk in enumerate(breakpoints):
        if brk and len(current) >= min_sentences:
            chunks.append(" ".join(current))
            current = [sentences[i + 1]]
        else:
            current.append(sentences[i + 1])

    if current:
        chunks.append(" ".join(current))

    # -------------------------------
    # Step 8: Display chunks
    # -------------------------------
    for i, chunk in enumerate(chunks):
        console.print(Panel(chunk, title=f"Chunk {i}", expand=False))

In [20]:
guarded_semantic_chunker(
    text=text
)

────────────────────────────── Level 4 — Guarded Semantic Chunking (Verbose & Fixed) ──────────────────────────────

                                                      Windows                                                      
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ W#  ┃ Sentences in Window                                                                                       ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ W0  │ MEDUSA                                                                                                    │
│     │ Jelly-fish                                                                                                │
│     │ Whole face puffed œdematous-eyes, nose, ears, lips. | Skin.––Numbness; burning, pricking heat.            │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W1  │ MEDUSA                                                                                                    │
│     │ Jelly-fish                                                                                                │
│     │ Whole face puffed œdematous-eyes, nose, ears, lips. | Skin.––Numbness; burning, pricking heat. |          │
│     │ Vesicular eruption especially on face, arms, shoulders, and breasts.                                      │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W2  │ Skin.––Numbness; burning, pricking heat. | Vesicular eruption especially on face, arms, shoulders, and    │
│     │ breasts. | Nettlerash (Apis; Chloral; Dulc).                                                              │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W3  │ Vesicular eruption especially on face, arms, shoulders, and breasts. | Nettlerash (Apis; Chloral; Dulc).  │
│     │ | Female.––Marked action on lacteal glands.                                                               │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W4  │ Nettlerash (Apis; Chloral; Dulc). | Female.––Marked action on lacteal glands. | The secretion of milk was │
│     │ established after lack of it in all previous confinements.                                                │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W5  │ Female.––Marked action on lacteal glands. | The secretion of milk was established after lack of it in all │
│     │ previous confinements. | Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep.      │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W6  │ The secretion of milk was established after lack of it in all previous confinements. |                    │
│     │ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. | MEL CUM SALE                │
│     │ Honey with Salt                                                                                           │
│     │ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of   │
│     │ the cervix.                                                                                               │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ W7  │ Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. | MEL CUM SALE                │
│     │ Honey with Salt                                                                                           │
│     │ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of   │
│     │ the cervix. | The special symptom leading to its

                                       Split Reasoning per Window Pair                                       
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Window Pair ┃ Similarity ┃ Cutoff ┃ Drop from prev ┃ Decision ┃ Reasoning                                 ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ W0 → W1     │     0.9766 │ 0.6086 │         0.0000 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W1 → W2     │     0.5997 │ 0.6086 │         0.3769 │ BREAK    │ below percentile cutoff, significant drop │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W2 → W3     │     0.7380 │ 0.6086 │        -0.1383 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W3 → W4     │     0.6870 │ 0.6086 │         0.0509 │ KEEP     │ significant drop                          │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W4 → W5     │     0.6364 │ 0.6086 │         0.0506 │ KEEP     │ significant drop                          │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W5 → W6     │     0.7824 │ 0.6086 │        -0.1460 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W6 → W7     │     0.8322 │ 0.6086 │        -0.0498 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W7 → W8     │     0.8080 │ 0.6086 │         0.0243 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W8 → W9     │     0.5250 │ 0.6086 │         0.2830 │ BREAK    │ below percentile cutoff, significant drop │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W9 → W10    │     0.8413 │ 0.6086 │        -0.3163 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W10 → W11   │     0.6497 │ 0.6086 │         0.1917 │ KEEP     │ significant drop                          │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W11 → W12   │     0.6352 │ 0.6086 │         0.0144 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W12 → W13   │     0.6219 │ 0.6086 │         0.0133 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W13 → W14   │     0.9296 │ 0.6086 │        -0.3077 │ KEEP     │ OK                                        │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W14 → W15   │     0.8157 │ 0.6086 │         0.1140 │ KEEP     │ significant drop                          │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W15 → W16   │     0.4000 │ 0.6086 │         0.4157 │ BREAK    │ below percentile cutoff, significant drop │
├─────────────┼────────────┼────────┼────────────────┼──────────┼───────────────────────────────────────────┤
│ W16 → W17   │     0.7278 │ 0.6086 │   

╭────────────────────────────────────────── Chunk 0 ───────────────────────────────────────────╮
│ MEDUSA                                                                                       │
│ Jelly-fish                                                                                   │
│ Whole face puffed œdematous-eyes, nose, ears, lips. Skin.––Numbness; burning, pricking heat. │
╰──────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 1 ────────────────────────────────────────────────────╮
│ Vesicular eruption especially on face, arms, shoulders, and breasts. Nettlerash (Apis; Chloral; Dulc).          │
│ Female.––Marked action on lacteal glands. The secretion of milk was established after lack of it in all         │
│ previous confinements. Relationship.––Compare: Pyrarara, Physalia (urticaria); Urtica, Homar, Sep. MEL CUM SALE │
│ Honey with Salt                                                                                                 │
│ Prolapsus uteri and chronic metritis, especially when associated with subinvolution and inflammation of the     │
│ cervix. The special symptom leading to its selection is a feeling of soreness across the hypogastrium from      │
│ ileum to ileum.                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 2 ────────────────────────────────────────────────────╮
│ Uterine displacements, and in the commencement of metritis Sensation as if bladder were too full. Pain from     │
│ sacrum towards pubes. Pain as if in ureters. Dose.––Third to sixth potency. Honey for itching of anus and       │
│ worms. METHYLENUM COERULEUM                                                                                     │
│ Aniline Dye                                                                                                     │
│ Methylene Blue                                                                                                  │
│ A remedy for neuralgia, neurasthenia, malaria; typhoid, here it diminishes the tympanites, delirium, and fever; │
│ pus infection. Tendency to tremor, chorea and epilepsy.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Chunk 3 ────────────────────────────────────────────────────╮
│ Nephritis (acute parenchymatous), scarlatinal nephritis. Urine acquires a green color. Bladder irritation from  │
│ its use antidoted by a little nutmeg. Surgical kidney with large amount of pus in urine. Gonorrhœal rheumatism  │
│ and cystitis. Backache, sciatica. Later states of apoplexy (Gisevius). Dose.––3x attenuation. A 2 per cent      │
│ solution locally, in chronic otitis with foul smelling discharge. A 1 per cent aqueous solution for ulcers and  │
│ abscesses of cornea.                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯